In [2]:
! pip install ipytest

     |████████████████████████████████| 280 kB 5.1 MB/s 
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: pytest
    Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import ipytest
ipytest.autoconfig(clean=False)

## 1. Refactoring
- 코드 구조나 가독성, 유연성 등의 개선을 목적으로 refactoring을 하게됨
- unit test가 없으면 정확하게 수행하기 어려울수 있음

#### required python modules

In [4]:
%%writefile constants.py


STATUS_ENDPOINT = "http://localhost:8080/mrstatus"

Writing constants.py


In [5]:
%%writefile mock_2.py


from datetime import datetime
import requests

from constants import STATUS_ENDPOINT


class BuildStatus:
    """The CI status of a pull request."""

    @staticmethod
    def build_date() -> str:
        return datetime.utcnow().isoformat()

    @classmethod
    def notify(cls, merge_request_id, status):
        build_status = {
            "id": merge_request_id,
            "status": status,
            "built_at": cls.build_date(),
        }
        response = requests.post(STATUS_ENDPOINT, json=build_status)
        response.raise_for_status()
        return response

Writing mock_2.py


#### test code

In [6]:
from datetime import datetime
from unittest import mock

import pytest

from constants import STATUS_ENDPOINT
from mock_2 import BuildStatus

In [7]:
%%ipytest -vv -rP --color=yes


@mock.patch("mock_2.requests")
def test_build_notification_sent(mock_requests):
    build_date = "2018-01-01T00:00:01"
    with mock.patch("mock_2.BuildStatus.build_date", return_value=build_date):
        BuildStatus.notify(123, "OK")

    expected_payload = {"id": 123, "status": "OK", "built_at": build_date}
    mock_requests.post.assert_called_with(
        STATUS_ENDPOINT, json=expected_payload
    )

======================================= test session starts ========================================
platform linux -- Python 3.7.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: typeguard-2.7.1
collecting ... collected 1 item

tmptvzapjcc.py::test_build_notification_sent PASSED                                          [100%]

============================================== PASSES ==============================================
======================================== 1 passed in 0.02s =========================================


### 1.1. Evolving our code
- 기존 코드는 `mock.patch`를 사용해서 unittest에서 제어하기 힘든 부분을 테스트 가능하도록 작성했었음
  - `requests.post`, `BuildStatus.build_date()`
- 이 코드는 아래와 같은 단점이 있음
  - module에 대한 정보를 포함하여 pull path를 줘야됨
  - refactoring할때 path가 변경되거나 name이 변경되면 관련 코드를 전부 수정해줘야 됨
- 위 문제는 `notify` method가 `requests` module에 직접 의존하고 있기 때문에 생긴 것으로 refactoring을 통해 이를 개선하려고 함

#### required python modules

In [8]:
%%writefile refactoring_1.py


from datetime import datetime

from constants import STATUS_ENDPOINT


class BuildStatus:

    endpoint = STATUS_ENDPOINT

    # transport를 넘겨주도록 수정
    # - requests 또는 Mock()을 넘겨줌
    # - post를 호출 가능하면 인터페이스는 문제 없음
    def __init__(self, transport):
        self.transport = transport

    @staticmethod
    def build_date() -> str:
        return datetime.utcnow().isoformat()

    def compose_payload(self, merge_request_id, status) -> dict:
        return {
            "id": merge_request_id,
            "status": status,
            "built_at": self.build_date(),
        }

    def deliver(self, payload):
        response = self.transport.post(self.endpoint, json=payload)
        response.raise_for_status()
        return response

    def notify(self, merge_request_id, status):
        return self.deliver(self.compose_payload(merge_request_id, status))


Writing refactoring_1.py


#### test code

In [9]:
from unittest.mock import Mock

import pytest

from refactoring_1 import BuildStatus

In [10]:
%%ipytest -vv -rP --color=yes


# 테스트 수행시 이 fixture를 넘겨줌
# - init transport argument: Mock transport 사용
# - build_date method: Mock 데이터를 사용
@pytest.fixture
def build_status():
    bstatus = BuildStatus(Mock())    
    bstatus.build_date = Mock(return_value="2018-01-01T00:00:01")    
    return bstatus


def test_build_notification_sent(build_status):

    build_status.notify(1234, "OK")

    expected_payload = {
        "id": 1234,
        "status": "OK",
        "built_at": build_status.build_date(),
    }
    
    build_status.transport.post.assert_called_with(
        build_status.endpoint, json=expected_payload
    )

======================================= test session starts ========================================
platform linux -- Python 3.7.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: typeguard-2.7.1
collecting ... collected 1 item

tmp05wfnjlx.py::test_build_notification_sent PASSED                                          [100%]

============================================== PASSES ==============================================
======================================== 1 passed in 0.02s =========================================


### 1.2. Production code isn't the only thing that evolves
- unittest 역시 production 코드 만큼 중요하다고 계속 강조해왔음
- 따라서 test code 역시 유지보수가 가능하고, 확장성이 충분하도록 설계할 필요성이 있음
- production code가 요구사항에 맞게 수정됨에 따라, test code도 이를 테스트할수 있도록 수정되어야 함
- `MergeRequest` 예제에서 `assert case`에 대해 재사용 가능한 abstraction을 작성하는 방식으로 refactoring 예정

#### required python modules

In [11]:
%%writefile mrstatus.py


from enum import Enum


class MergeRequestStatus(Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    PENDING = "pending"


class MergeRequestExtendedStatus(Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    PENDING = "pending"
    OPEN = "open"
    CLOSED = "closed"


class MergeRequestException(Exception):
    """Something went wrong with the merge request."""


Writing mrstatus.py


In [12]:
%%writefile refactoring_2.py


from mrstatus import MergeRequestExtendedStatus, MergeRequestException


class AcceptanceThreshold:
    def __init__(self, merge_request_context: dict) -> None:
        self._context = merge_request_context

    def status(self):
        if self._context["downvotes"]:
            return MergeRequestExtendedStatus.REJECTED
        elif len(self._context["upvotes"]) >= 2:
            return MergeRequestExtendedStatus.APPROVED
        return MergeRequestExtendedStatus.PENDING


class MergeRequest:
    def __init__(self):
        self._context = {"upvotes": set(), "downvotes": set()}
        self._status = MergeRequestExtendedStatus.OPEN

    def close(self):
        self._status = MergeRequestExtendedStatus.CLOSED

    @property
    def status(self):
        if self._status == MergeRequestExtendedStatus.CLOSED:
            return self._status

        return AcceptanceThreshold(self._context).status()

    def _cannot_vote_if_closed(self):
        if self._status == MergeRequestExtendedStatus.CLOSED:
            raise MergeRequestException("can't vote on a closed merge request")

    def upvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)

    def downvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].add(by_user)


Writing refactoring_2.py


#### test code

In [13]:
from unittest import TestCase, main

from refactoring_2 import (AcceptanceThreshold, MergeRequest,
                           MergeRequestException, MergeRequestExtendedStatus)

In [14]:
%%ipytest -vv -rP --color=yes


class TestMergeRequestStatus(TestCase):
    def setUp(self):
        self.merge_request = MergeRequest()

    def assert_rejected(self):
        self.assertEqual(
            self.merge_request.status, MergeRequestExtendedStatus.REJECTED
        )

    def assert_pending(self):
        self.assertEqual(
            self.merge_request.status, MergeRequestExtendedStatus.PENDING
        )

    def assert_approved(self):
        self.assertEqual(
            self.merge_request.status, MergeRequestExtendedStatus.APPROVED
        )

    def test_simple_rejected(self):
        self.merge_request.downvote("maintainer")
        self.assert_rejected()

    def test_just_created_is_pending(self):
        self.assert_pending()

    def test_pending_awaiting_review(self):
        self.merge_request.upvote("core-dev")
        self.assert_pending()

    def test_approved(self):
        self.merge_request.upvote("dev1")
        self.merge_request.upvote("dev2")
        self.assert_approved()

    def test_no_double_approve(self):
        self.merge_request.upvote("dev1")
        self.merge_request.upvote("dev1")
        self.assert_pending()

    def test_upvote_changes_to_downvote(self):
        self.merge_request.upvote("dev1")
        self.merge_request.upvote("dev2")
        self.merge_request.downvote("dev1")

        self.assert_rejected()

    def test_downvote_to_upvote(self):
        self.merge_request.upvote("dev1")
        self.merge_request.downvote("dev2")
        self.merge_request.upvote("dev2")

        self.assert_approved()

    def test_invalid_types(self):
        merge_request = MergeRequest()
        self.assertRaises(TypeError, merge_request.upvote, {"invalid-object"})

    def test_cannot_vote_on_closed_merge_request(self):
        merge_request = MergeRequest()
        merge_request.close()
        self.assertRaises(MergeRequestException, merge_request.upvote, "dev1")
        self.assertRaisesRegex(
            MergeRequestException,
            "can't vote on a closed merge request",
            merge_request.downvote,
            "dev1",
        )


class TestAcceptanceThreshold(TestCase):
    def setUp(self):
        self.fixture_data = (
            (
                {"downvotes": set(), "upvotes": set()},
                MergeRequestExtendedStatus.PENDING,
            ),
            (
                {"downvotes": set(), "upvotes": {"dev1"}},
                MergeRequestExtendedStatus.PENDING,
            ),
            (
                {"downvotes": "dev1", "upvotes": set()},
                MergeRequestExtendedStatus.REJECTED,
            ),
            (
                {"downvotes": set(), "upvotes": {"dev1", "dev2"}},
                MergeRequestExtendedStatus.APPROVED,
            ),
        )

    def test_status_resolution(self):
        for context, expected in self.fixture_data:
            with self.subTest(context=context):
                status = AcceptanceThreshold(context).status()
                self.assertEqual(status, expected)

======================================= test session starts ========================================
platform linux -- Python 3.7.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: typeguard-2.7.1
collecting ... collected 11 items

tmpe2kmfwtb.py::TestMergeRequestStatus::test_approved PASSED                                 [  9%]
tmpe2kmfwtb.py::TestMergeRequestStatus::test_cannot_vote_on_closed_merge_request PASSED      [ 18%]
tmpe2kmfwtb.py::TestMergeRequestStatus::test_downvote_to_upvote PASSED                       [ 27%]
tmpe2kmfwtb.py::TestMergeRequestStatus::test_invalid_types PASSED                            [ 36%]
tmpe2kmfwtb.py::TestMergeRequestStatus::test_just_created_is_pending PASSED                  [ 45%]
tmpe2kmfwtb.py::TestMergeRequestStatus::test_no_double_approve PASSED                        [ 54%]
tmpe2kmfwtb.py::TestMergeRequestStatus::test_pending_awaiting_review PASSED                  [ 63%]
tmpe2k